In [4]:
##############################
# Solves -u'' + u = f        #
# With zero Neumann boundary #
##############################
import radiant as rad
import numpy as np


# Problem Parameters
a = 0
b = 2 * np.pi


def u(x):
    return np.cos(x * 2 * np.pi / (b - a))


def f(x):
    return ((2 * np.pi / (b - a)) ** 2 + 1) * np.cos(x * 2 * np.pi / (b - a))


In [ ]:
# Fix delta, vary N
# Parameters
d = 1
k = 1
delta = 1.

Ns = np.linspace(10, 100, 10, dtype=int)
results = np.zeros((Ns.size, 2))

for i, N in enumerate(Ns):
    phi, points, A, fs = rad.generate(a, b, f, N, d, k, delta)
    
    # Solve for approximate solution
    u_approx, alphas = rad.solve(phi, points, A, fs)
    error = rad.error(u, u_approx, a, b)
    cond = np.linalg.cond(A)
    
    results[i,:] = [error, cond]
    
    print("Trial:", i)
    print("  N:", N)
    print("  L2 Relative Error:", error)
    print("  System condition number:", cond)

import matplotlib.pyplot as plt
fig, axs = plt.subplots(1, 2)

axs[0].set_xlabel("No. Centres")
axs[0].set_ylabel(r"$L^{2}$ Relative Error")
axs[0].plot(Ns, results[:, 0])

axs[1].set_xlabel("No. Centres")
axs[1].set_ylabel("Matrix Condition Number")
axs[1].plot(Ns, results[:, 1])

plt.show()

# TODO:
# Look at cond number
#   - Fix delta change N
#   - Fix N look at delta
# Multilevel algorithm for plain vanilla function approx


Trial: 0
  N: 10
  L2 Relative Error: 0.7108288841750018
  System condition number: 16.85359605669534
Trial: 1
  N: 20
  L2 Relative Error: 0.6665511041786354
  System condition number: 43.488206035588156
Trial: 2
  N: 30
  L2 Relative Error: 0.6603614344744714
  System condition number: 162.01852174049202
Trial: 3
  N: 40
  L2 Relative Error: 0.657839576689004
  System condition number: 616.6214086730715
Trial: 4
  N: 50
  L2 Relative Error: 0.656537210882488
  System condition number: 1699.3038354580492
Trial: 5
  N: 60
  L2 Relative Error: 0.6555896211242704
  System condition number: 4690.773113461524
Trial: 6
  N: 70
  L2 Relative Error: 0.6551004016972233
  System condition number: 9343.45046228549
